<a href="https://colab.research.google.com/github/nicholaspearson43/Homework-Algorithmic-Design/blob/main/Pearson_HW3_AD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithmic Design - Homework 3
## Nicholas Andrea Pearson - SM3500457

I decided to use the binheap implementation we created during the lectures.
A few adjustments were made to better fit the requirements of the queue for point 1.

In [1]:
from typing import TypeVar, Generic, Union, List
from numbers import Number

T = TypeVar('T')

def min_order(a: Number, b: Number) -> bool:
    return a<= b

def max_order(a: Number, b: Number) -> bool:
    return a>=b

class binheap(Generic[T]):
    LEFT = 0
    RIGHT=1
    def __init__(self, A: Union[int, List[T]], total_order=None):
        if total_order is None:
            self._torder=min_order
        else:
            self._torder=total_order

        if isinstance(A, int):
            self._size=0
            self._A=[None]*A
        else:
            self._size = len(A)
            self._A = A

        self._build_heap()

    @staticmethod
    def parent(node: int) -> Union[int, None]:
        if node == 0:
            return None
        return (node-1)//2

    @staticmethod
    def child(self, node:int, side:int)-> int:
        return 2*node+1+side

    @staticmethod
    def left(node: int) -> int:
        return 2*node+1

    @staticmethod
    def right(node: int) -> int:
        return 2 * node + 2

    def __len__(self):
        return self._size

    def is_empty(self) -> bool:
        return self._size==0


    def _swap_keys(self, node_a:int, node_b: int) -> None:
        tmp = self._A[node_a]
        self._A[node_a]=self._A[node_b]
        self._A[node_b]=tmp

        tmp = self._A[node_a].index_in_heap
        self._A[node_a].index_in_heap=self._A[node_b].index_in_heap
        self._A[node_b].index_in_heap=tmp

    def _heapify(self, node: int) -> None:
        keep_fixing=True

        while keep_fixing:
            min_node = node
            for child_idx in [binheap.left(node), binheap.right(node)]:
                 if (child_idx<self._size and self._torder(self._A[child_idx].d, self._A[min_node].d)):
                    min_node=child_idx
            
            if min_node != node:
                self._swap_keys(min_node, node)
                node=min_node
            else:
                keep_fixing=False

    def remove_minimum(self) -> T:
        if self.is_empty():
            raise RuntimeError("The heap is empty")

        self._swap_keys(0, self._size-1)
        #self._A[0] = self._A[self._size-1]


        self._size = self._size - 1
        self._heapify(0)

        return self._A[self._size]

    def _build_heap(self) -> None:
        for i in range((self._size-1), -1, -1):
            self._heapify(i)

    def decrease_key(self, node: int, new_value: T) -> None:
        #if self._torder(self._A[node].d, new_value):
        #    raise RuntimeError(f'{new_value} is not smaller than'+f'{self.A[node]}')

        self._A[node].d= new_value
        parent = binheap.parent(node)
        while (node != 0 and not self._torder(self._A[parent].d, self._A[node].d)):
            self._swap_keys(node, parent)

            node=parent
            parent=binheap.parent(node)

    def insert(self, value: T) -> None:
        if self._size >= len(self._A):
            raise RuntimeError('The Heap is full')

        if self.is_empty():
            self._A[0]=value
            self._size += 1
        else:
            parent=binheap.parent(self._size)
            if self._torder(self._A[parent].d, value):
                self._A[self._size].d=value
                self._size += 1
            else:
                self._A[self._size].d= self._A[parent].d
                self._size += 1
                self.decrease_key(self._size - 1, value)


    def __repr__(self) -> str:
        bt_str = ''

        next_node = 1
        up_to = 2

        while next_node <= self._size:
            level = '\t'.join(f'{v}' for v in self._A[next_node-1: up_to-1])

            if next_node == 1:
                bh_str = level
            else:
                bh_str += f'\n{level}'

            next_node = up_to
            up_to = 2*up_to

        return bh_str



## Exercise 1 
Implement the binary heap-based version of the Dijkstra's algorithm.

### Solution:

The first thing we need to do is implement a weighted graph class with nodes and edges.
The attribute ```importance``` was added to define an order in which to remove the nodes as required in exercise ```2.1```.




In [2]:
class Node:
  def __init__(self, key: int, importance:int = 1):
    self.key = key 
    self.d = float('inf') #for Dijkstra
    self.pred = None #for Dijkstra
    self.importance = importance # the higher this value, the more important the node. Used in part 2
    self.index_in_heap = None #for Dijkstra
    self.index_in_adj = 0

  def print_node(self):
      print(f" The key of the node is: {self.key}")

In [3]:
class Edge:
    def __init__(self, head: Node, tail: Node, weight = 1):
      self.head = head
      self.tail = tail
      self.weight = weight

In [4]:
class W_Graph:
  def __init__(self, nodes: List[Node], edges: List[Edge]):
      self.V = nodes
      self.E = edges
      self.A = [[] for node in range(len(self.V))]
      
      for i in range(len(self.V)):
        self.V[i].index_in_adj = i
        for edge in self.E:
          if edge.head.key == self.V[i].key:
            self.A[i].append((edge.tail, edge.weight))



  def print_A(self): #auxilliary function to better visualize the graph
    out = []
    for i in range(len(self.V)):
      out.append(self.V[i].key)
    if(len(out) != 0 and len(out) != 1):
      print(f"The remaining nodes are: {out}.")
      print("The graph has the following connenctions:")
    elif (len(out) == 1):
      print(f"The only remaining node is: {out}.")
      print("Since there is only one node, there are no edges.")
    elif (len(out) == 0):
      print("There are no nodes and thus no edges.")
    for i in self.V:
      for j in self.E:
        if i.key == j.head.key:
          print(f"[Head key {i.key} | Tail key {j.tail.key} | Weight {j.weight}]")
    print("\n")
          

In [5]:
a=Node(0) #if the importance is not specified it is set to 1 by default.
b=Node(1, 2)
c=Node(2, 3)
d=Node(3, 4)
e=Node(4, 5)
f=Node(5, 6)
g=Node(6, 7)
h=Node(7, 8)

In [6]:
nodes = [a,b,c,d,e,f,g,h]
edges=[Edge(a,e,1),Edge(a,f,1),Edge(b,c,2),Edge(c,b,1),Edge(c,d,3),Edge(d,h,3),Edge(d,g,1),Edge(e,f,1),Edge(e,a,3),Edge(f,h,1),Edge(g,h,1),Edge(h,g,1),Edge(h,a,1) ]
WG = W_Graph(nodes, edges)
WG.print_A()

The remaining nodes are: [0, 1, 2, 3, 4, 5, 6, 7].
The graph has the following connenctions:
[Head key 0 | Tail key 4 | Weight 1]
[Head key 0 | Tail key 5 | Weight 1]
[Head key 1 | Tail key 2 | Weight 2]
[Head key 2 | Tail key 1 | Weight 1]
[Head key 2 | Tail key 3 | Weight 3]
[Head key 3 | Tail key 7 | Weight 3]
[Head key 3 | Tail key 6 | Weight 1]
[Head key 4 | Tail key 5 | Weight 1]
[Head key 4 | Tail key 0 | Weight 3]
[Head key 5 | Tail key 7 | Weight 1]
[Head key 6 | Tail key 7 | Weight 1]
[Head key 7 | Tail key 6 | Weight 1]
[Head key 7 | Tail key 0 | Weight 1]




![picture](https://drive.google.com/uc?export=view&id=1ul9_8a4zF_W18aWpetiXRPE7_o40UiYM)

In [7]:
def init_SSSP(G: W_Graph):
  for v in G.V:
    v.d = float('inf')
    v.prec = None

In [8]:
def relax(Q: binheap, u: Node, v:Node, w: float):
  if (u.d + w < v.d):
    Q.decrease_key(v.index_in_heap, u.d+w)
    v.pred = u

In [9]:
def path_finder(i: Node, s: Node): #returns the shortest path between two nodes
  path = []
  if i.key == s.key:
    return path
  else:
    while i.key != s.key:
      path.append(i.key)
      i = i.pred
    path.append(s.key)
    final_path = path[::-1]
    return final_path

In [10]:
def dijkstra(G: W_Graph, s: Node, d: Node = None): #the second node is optional, if it is inserted the function only returns 
  init_SSSP(G)                                     #the shortest path between the source node and the destination node and its length.
  s.d = 0                                          #if it is not inserted it returns the shortest path from the source node to all reachable nodes
                                                   #and their length
  Q = binheap(G.V)
  for i, node in enumerate(Q._A): 
    node.index_in_heap = i

  results = []
  while not Q.is_empty():
    u = Q.remove_minimum()
    for element in G.A[u.index_in_adj]:
      v, w = element
      relax(Q, u, v, w)


    if d == None:
      if u == s:
        print(f"The path from the source node with key {s.key} to itself has length 0")
      else:
        if u.d != float('inf'):
          print(f"The path from the node with key {s.key} to {u.key} has length {u.d}")
          print("The path visits the following nodes:")
          print(f"{path_finder(u, s)}")
          results.append((path_finder(u, s),u.d))
      print("\n")
    else:
      if u == d and u.d != float('inf'):
        print(f"The path from the node with key {s.key} to {u.key} has length {u.d}")
        print("The path visits the following nodes:")
        print(f"{path_finder(u, s)}")
        print("\n")
        results.append((path_finder(u, s),u.d))
        break
  if len(results) == 0:
    print("There is no path between the two nodes")
    return results, float('inf')
  return results


In [11]:
dijkstra(WG, d, f)

The path from the node with key 3 to 5 has length 4
The path visits the following nodes:
[3, 6, 7, 0, 5]




[([3, 6, 7, 0, 5], 4)]

In [12]:
dijkstra(WG, d)

The path from the source node with key 3 to itself has length 0


The path from the node with key 3 to 6 has length 1
The path visits the following nodes:
[3, 6]


The path from the node with key 3 to 7 has length 2
The path visits the following nodes:
[3, 6, 7]


The path from the node with key 3 to 0 has length 3
The path visits the following nodes:
[3, 6, 7, 0]


The path from the node with key 3 to 4 has length 4
The path visits the following nodes:
[3, 6, 7, 0, 4]


The path from the node with key 3 to 5 has length 4
The path visits the following nodes:
[3, 6, 7, 0, 5]








[([3, 6], 1),
 ([3, 6, 7], 2),
 ([3, 6, 7, 0], 3),
 ([3, 6, 7, 0, 4], 4),
 ([3, 6, 7, 0, 5], 4)]

In [13]:
dijkstra(WG, d, b)

There is no path between the two nodes


([], inf)

## Exercise 2 
Consider the contraction hierarchies presented during the course. Assume to deal with graphs that can be fully represented in the memory of your computer. Implement:
1. an algorithm to add the shortcuts to a graph;
2. a bidirectional version of Dijkstra algorithm that can operate on the graphs decorated by the algorithm at Point 2a. <re>

### Solution:
#### 2.1

In [14]:
def shortcut(G: W_Graph, imp: int = None):
  pres = bool(False)
  if imp == None:  #searches for the node with the specified importance, if it has not been inserted it searches for the node with 
    imp = min(node.importance for node in G.V) #the lowest importance

  else:    
    for i in range(len(G.V)):
      if G.V[i].importance == imp:
        pres = True
        break

  for i in range(len(G.V)):
    if G.V[i].importance == imp:
      k = i
      break

  if (pres == False):
    print("You either didn't insert a specific importance or the desired importance is not associted to any of the nodes. \n Searching for the node with least importance") 

  koi = G.V[k].key
  nodes1 = []
  print(f"The eliminated node is the one with key: {koi}")

  for i in range(len(G.V)): #creates the list of nodes for the new graph
    if not G.V[i].key == koi:
      nodes1.append(G.V[i])

  edges1 = []
  tmp=[]
  for i in range(len(G.E)):
    if  G.E[i].head.key == koi or G.E[i].tail.key == koi: 
      tmp.append(G.E[i])
    else:
      edges1.append(G.E[i])  #adds the edges that do not involve the node we want to remove


  test_in = bool(False)  #checks if there are edges which enter the node we want to remove
  test_out = bool(False) #checks if there are edges which leave the node we want to remove
  in_list = []           #if there is not at least one edge going in and one going out we can remove the node without creating any new edges.
  out_list = []

  for i in range(len(G.E)):
    if G.E[i].head.key == koi:
      test_out = True
      out_list.append(G.E[i])
    if G.E[i].tail.key == koi:
      test_in = True
      in_list.append(G.E[i])

  new_edges = []
  if not (test_in == True and test_out == True): #as we do not have to create any new edges we can create a new graph.
    G_New = W_Graph(nodes1, edges1)
  else:
    for begin in in_list:
      for end in out_list:
        tmp = Edge(begin.head, end.tail, begin.weight+end.weight) #creates all possible combinations of edges among the nodes connected to the node 
        new_edges.append(tmp)                                     #we want to remove without going through the node itself.
                                                                  #if we have 3 nodes a, b, c with an edge going from a to b and one from b to c
                                                                  #and we want to remove node b, we would obtain an edge going directly from a to c
  
  new_edges_2 = [edge for edge in new_edges if not edge.head.key == edge.tail.key] #if the starting node and the ending node are the same we
                                                                                   #remove the edge
  to_add = []
  is_dup = bool(False)
  
  for n in new_edges_2:
    for o in edges1:
      if(n.head == o.head and n.tail == o.tail): #checks is there already is an edge between the two nodes and compares the two weights
        if o.weight > n.weight:                  #keeps the edge with the lowest weight
          edges1.remove(o)
        else:
          new_edges_2.remove(n)

  to_add = new_edges_2 + edges1
  
  G_New = W_Graph(nodes1, to_add)    #creates a new graph

  return G_New

In [15]:
def hierarchy(W: W_Graph):
  it = len(W.V)
  W.print_A()
  for i in range(it):
    W = shortcut(W)
    W.print_A() 

In [16]:
hierarchy(WG)

The remaining nodes are: [2, 1, 5, 4, 0, 7, 6, 3].
The graph has the following connenctions:
[Head key 2 | Tail key 1 | Weight 1]
[Head key 2 | Tail key 3 | Weight 3]
[Head key 1 | Tail key 2 | Weight 2]
[Head key 5 | Tail key 7 | Weight 1]
[Head key 4 | Tail key 5 | Weight 1]
[Head key 4 | Tail key 0 | Weight 3]
[Head key 0 | Tail key 4 | Weight 1]
[Head key 0 | Tail key 5 | Weight 1]
[Head key 7 | Tail key 6 | Weight 1]
[Head key 7 | Tail key 0 | Weight 1]
[Head key 6 | Tail key 7 | Weight 1]
[Head key 3 | Tail key 7 | Weight 3]
[Head key 3 | Tail key 6 | Weight 1]


You either didn't insert a specific importance or the desired importance is not associted to any of the nodes. 
 Searching for the node with least importance
The eliminated node is the one with key: 0
The remaining nodes are: [2, 1, 5, 4, 7, 6, 3].
The graph has the following connenctions:
[Head key 2 | Tail key 1 | Weight 1]
[Head key 2 | Tail key 3 | Weight 3]
[Head key 1 | Tail key 2 | Weight 2]
[Head key 5 | Tail key

#### 2.2

In [17]:
a=Node(0, 1)
b=Node(1, 2)
c=Node(2, 3)
d=Node(3, 4)
e=Node(4, 5)
f=Node(5, 6)
g=Node(6, 7)
h=Node(7, 8)

nodes = [a,b,c,d,e,f,g,h]
edges=[Edge(a,e,1),Edge(a,f,1),Edge(b,c,2),Edge(c,b,1),Edge(c,d,3),Edge(d,h,3),Edge(d,g,1),Edge(e,f,1),Edge(e,a,3),Edge(f,h,1),Edge(g,h,1),Edge(h,g,1),Edge(h,a,1) ]
WG = W_Graph(nodes, edges)
WG.print_A()

The remaining nodes are: [0, 1, 2, 3, 4, 5, 6, 7].
The graph has the following connenctions:
[Head key 0 | Tail key 4 | Weight 1]
[Head key 0 | Tail key 5 | Weight 1]
[Head key 1 | Tail key 2 | Weight 2]
[Head key 2 | Tail key 1 | Weight 1]
[Head key 2 | Tail key 3 | Weight 3]
[Head key 3 | Tail key 7 | Weight 3]
[Head key 3 | Tail key 6 | Weight 1]
[Head key 4 | Tail key 5 | Weight 1]
[Head key 4 | Tail key 0 | Weight 3]
[Head key 5 | Tail key 7 | Weight 1]
[Head key 6 | Tail key 7 | Weight 1]
[Head key 7 | Tail key 6 | Weight 1]
[Head key 7 | Tail key 0 | Weight 1]




In [18]:
from copy import deepcopy

def flip_graph(G: W_Graph) -> W_Graph: #function to invert the edges of a graph
  old_nodes = G.V
  new_nodes = deepcopy(G.V) #a deepcopy is required to create two separate versions of the nodes  

  old_edges = G.E
  new_edges = []

  for o in old_edges: #flippes the edges
    ho = o.head.key
    to = o.tail.key
    for n in new_nodes:
      if ho == n.key:
        tn = n
      if to == n.key:
        hn = n
    ne = Edge(hn, tn, o.weight )
    new_edges.append(ne)
  
  FG = W_Graph(new_nodes, new_edges) #creates new graph
  return FG

In [19]:
def init_SSSP_bi(G: W_Graph, n: Node): #it is used to directly initialize the source nodes distance to zero
  for v in G.V:
    if v.key == n.key:
      v.d = 0
    else:
      v.d = float('inf')
    v.prec = None

In [20]:
def bijkstra(G: W_Graph, s: Node, d: Node):
  flipped_G = flip_graph(G)       #creates a new graph with the same nodes as the previous but inverted edges 
    
  init_SSSP_bi(G, s)
  init_SSSP_bi(flipped_G, d)

  fin = []
  flipped_fin = []

  Q = binheap(G.V)                
  flipped_Q = binheap(flipped_G.V)

  for i, node in enumerate(Q._A):
    node.index_in_heap = i

  for i, node in enumerate(flipped_Q._A):
    node.index_in_heap = i

    
  while not Q.is_empty() and not flipped_Q.is_empty():
    u = Q.remove_minimum()
    u_f = flipped_Q.remove_minimum()

    for element in G.A[u.index_in_adj]:
      v, w = element
      relax(Q, u, v, w)
      

    for element in flipped_G.A[u_f.index_in_adj]:
      v_f, w_f = element
      relax(flipped_Q, u_f, v_f, w_f)
      

    fin.append(u)
    flipped_fin.append(u_f)

    for i in fin:                     #a similar procedure to the one used in the normal version of Dijkstra is applied in both directions
      if u_f.key == i.key:            #the procedure is stopped when the same node is finalized in both  direcions
        if u_f.d+i.d != float('inf'):
          print(f"The shortest path between the node with key {s.key} to the node with key {d.key} has length {u_f.d + i.d}:")
          print("The path visits the following nodes:")
          print(path_finder(i, s)[:-1]+path_finder(u_f, d)[::-1])
          return path_finder(i, s)[:-1]+path_finder(u_f, d)[::-1], u_f.d + i.d

    for j in flipped_fin:
      if u.key == j.key:
        if u.d+j.d != float('inf'):
          print(f"The shortest path between the node with key {s.key} to the node with key {d.d} has length {u.d + j.d}:")
          print("The path visits the following nodes:")
          print(path_finder(u, s)[:-1]+path_finder(j, d)[::-1])
          return path_finder(u, s)[:-1]+path_finder(j, d)[::-1], u.d + j.d
  print("There is no path between the two nodes")
  return [], float('inf')


In [21]:
bijkstra(WG, e, b)

There is no path between the two nodes


([], inf)

In [22]:
bijkstra(WG, b, e)

The shortest path between the node with key 1 to the node with key 4 has length 9:
The path visits the following nodes:
[1, 2, 3, 6, 7, 0, 4]


([1, 2, 3, 6, 7, 0, 4], 9)